In [ ]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn
import seaborn as sns
from scipy import stats
from scipy.stats import norm,skew,kurtosis
import matplotlib.pyplot as plt
%matplotlib auto

In [ ]:
import numpy as np
from collections import Counter

def train_data_generation(filename):
    train_file = open(filename, encoding= 'iso 8859-15')
    train_data_raw = np.loadtxt(train_file, delimiter= ',', dtype = str, skiprows = 1)

    n_parameters = 18
    n_date = len(Counter(train_data_raw[:, 0]).keys())

    train_data_raw[train_data_raw == 'NR'] = 0

    train_data_raw = train_data_raw[:, 3:].astype(np.float64)

    train_data = train_data_raw[:n_parameters, :]

    for i in range(1, n_date):
        temp = train_data_raw[(n_parameters * (i - 1)): (n_parameters * i), :]
        train_data = np.hstack((train_data, temp))
        
    orig_data=train_data.copy()
    
    n_series = 9
    target_row = 9

    mean_x = np.mean(train_data, axis = 1)
    std_x = np.std(train_data, axis = 1)

#     for i in range(n_parameters):
#         if std_x[i] != 0:
#             train_data[i, :] = (train_data[i, :] - mean_x[i])/std_x[i]

    final_data = []


    for j in range(train_data.shape[1] - n_series):
        key = train_data[:, j:j+n_series].reshape(18*9, )
        value = train_data[target_row, j+n_series]
        final_data.append([key, value])

    return orig_data,final_data, [mean_x, std_x]


In [3]:
train_file = r'F:\study\ml\HonyiLee2020\hw1\data\train.csv'
orig_data,train_data, [mean_x, std_x] = train_data_generation(train_file)

In [4]:
len(train_data)

5751

In [5]:
x=np.empty((len(train_data),162))
y=np.empty((len(train_data),1))

In [6]:
for i in range(len(train_data)):
    x[i,:]=train_data[i][0].reshape(1,-1)
    y[i,:]=train_data[i][1]

In [7]:
x.shape

(5751, 162)

In [8]:
y.shape

(5751, 1)

In [9]:
x[0]

array([1.40e+01, 1.40e+01, 1.40e+01, 1.30e+01, 1.20e+01, 1.20e+01,
       1.20e+01, 1.20e+01, 1.50e+01, 1.80e+00, 1.80e+00, 1.80e+00,
       1.80e+00, 1.80e+00, 1.80e+00, 1.80e+00, 1.80e+00, 1.80e+00,
       5.10e-01, 4.10e-01, 3.90e-01, 3.70e-01, 3.50e-01, 3.00e-01,
       3.70e-01, 4.70e-01, 7.80e-01, 2.00e-01, 1.50e-01, 1.30e-01,
       1.20e-01, 1.10e-01, 6.00e-02, 1.00e-01, 1.30e-01, 2.60e-01,
       9.00e-01, 6.00e-01, 5.00e-01, 1.70e+00, 1.80e+00, 1.50e+00,
       1.90e+00, 2.20e+00, 6.60e+00, 1.60e+01, 9.20e+00, 8.20e+00,
       6.90e+00, 6.80e+00, 3.80e+00, 6.90e+00, 7.80e+00, 1.50e+01,
       1.70e+01, 9.80e+00, 8.70e+00, 8.60e+00, 8.50e+00, 5.30e+00,
       8.80e+00, 9.90e+00, 2.20e+01, 1.60e+01, 3.00e+01, 2.70e+01,
       2.30e+01, 2.40e+01, 2.80e+01, 2.40e+01, 2.20e+01, 2.10e+01,
       5.60e+01, 5.00e+01, 4.80e+01, 3.50e+01, 2.50e+01, 1.20e+01,
       4.00e+00, 2.00e+00, 1.10e+01, 2.60e+01, 3.90e+01, 3.60e+01,
       3.50e+01, 3.10e+01, 2.80e+01, 2.50e+01, 2.00e+01, 1.90e

In [10]:
y[0]

array([30.])

In [11]:
x.shape

(5751, 162)

In [12]:
x1=pd.DataFrame(x)

In [13]:
x1.T.describe()

,0,1,2,3,4,5,6,7,8,9,...,5741,5742,5743,5744,5745,5746,5747,5748,5749,5750
count,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,...,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000
mean,20.772901,21.665864,24.067284,27.374568,30.456914,33.458457,36.463457,39.582778,42.942099,43.143827,...,21.074938,21.339753,21.037840,20.971111,20.359136,20.204198,19.997716,20.378642,20.380370,20.110926
std,34.045272,35.319302,43.765793,51.725311,59.289896,66.394884,72.542403,77.930304,82.624760,81.482061,...,27.024134,27.530209,27.053115,27.453030,27.030013,27.142983,27.282312,28.336946,29.104168,29.150311
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.425000,1.500000,1.525000,1.600000,1.800000,1.800000,1.800000,1.800000,1.800000,1.800000,...,1.600000,1.700000,1.625000,1.700000,1.625000,1.700000,1.600000,1.625000,1.600000,1.600000
50%,3.150000,4.550000,4.800000,5.200000,5.200000,5.200000,6.750000,7.850000,11.000000,11.000000,...,7.000000,6.850000,6.850000,6.450000,5.950000,5.400000,5.100000,4.550000,3.800000,3.800000
75%,24.750000,26.500000,26.500000,28.000000,28.750000,29.750000,36.750000,39.500000,42.000000,44.000000,...,32.500000,32.500000,32.500000,32.500000,28.000000,27.750000,26.750000,25.750000,25.000000,24.750000
max,232.000000,232.000000,283.000000,283.000000,297.000000,316.000000,316.000000,316.000000,316.000000,316.000000,...,111.000000,111.000000,100.000000,100.000000,100.000000,100.000000,100.000000,106.000000,106.000000,106.000000


In [14]:
orig_data.shape

(18, 5760)

In [15]:
orig_data[:10,:]

array([[14.  , 14.  , 14.  , ..., 14.  , 14.  , 15.  ],
       [ 1.8 ,  1.8 ,  1.8 , ...,  1.7 ,  1.7 ,  1.7 ],
       [ 0.51,  0.41,  0.39, ...,  0.26,  0.2 ,  0.19],
       ...,
       [16.  , 30.  , 27.  , ..., 22.  , 32.  , 32.  ],
       [56.  , 50.  , 48.  , ..., 36.  , 24.  , 29.  ],
       [26.  , 39.  , 36.  , ...,  0.  ,  7.  ,  7.  ]])

In [16]:
train_data[0]

[array([1.40e+01, 1.40e+01, 1.40e+01, 1.30e+01, 1.20e+01, 1.20e+01,
        1.20e+01, 1.20e+01, 1.50e+01, 1.80e+00, 1.80e+00, 1.80e+00,
        1.80e+00, 1.80e+00, 1.80e+00, 1.80e+00, 1.80e+00, 1.80e+00,
        5.10e-01, 4.10e-01, 3.90e-01, 3.70e-01, 3.50e-01, 3.00e-01,
        3.70e-01, 4.70e-01, 7.80e-01, 2.00e-01, 1.50e-01, 1.30e-01,
        1.20e-01, 1.10e-01, 6.00e-02, 1.00e-01, 1.30e-01, 2.60e-01,
        9.00e-01, 6.00e-01, 5.00e-01, 1.70e+00, 1.80e+00, 1.50e+00,
        1.90e+00, 2.20e+00, 6.60e+00, 1.60e+01, 9.20e+00, 8.20e+00,
        6.90e+00, 6.80e+00, 3.80e+00, 6.90e+00, 7.80e+00, 1.50e+01,
        1.70e+01, 9.80e+00, 8.70e+00, 8.60e+00, 8.50e+00, 5.30e+00,
        8.80e+00, 9.90e+00, 2.20e+01, 1.60e+01, 3.00e+01, 2.70e+01,
        2.30e+01, 2.40e+01, 2.80e+01, 2.40e+01, 2.20e+01, 2.10e+01,
        5.60e+01, 5.00e+01, 4.80e+01, 3.50e+01, 2.50e+01, 1.20e+01,
        4.00e+00, 2.00e+00, 1.10e+01, 2.60e+01, 3.90e+01, 3.60e+01,
        3.50e+01, 3.10e+01, 2.80e+01, 2.50e+01, 

In [17]:
w_shape = (1, 18*9)

#get training data set and validation set, randomly chosen.
np.random.shuffle(train_data)
test_data = train_data[:500]
train_data = train_data[500:]

#set initial values
w = np.zeros(w_shape, dtype = float)
b = 0
eta = 0.0006 #learning rate

n_iter = 1000 #iteration time
train_loss = 0
test_loss = 0
result = []

for i in range(n_iter):
    train_loss = 0
    for j in range(len(train_data)):
        x = train_data[j][0]
        y = np.dot(w, x) + b
        target = train_data[j][1]

        w_gradient = 2 * (y - target) * x.T
        b_gradient = 2 * (y - target)

        w = w - eta * w_gradient
        b = b - eta * b_gradient

        train_loss += (target - y) ** 2

    result.append(train_loss)
np.savetxt('train_loss.txt', result)

for k in range(len(test_data)):

    x = test_data[k][0]
    y = np.dot(w, x) + b
    target = test_data[k][1]

    test_loss += (target - y) ** 2

print(test_loss/5)

x_train = []
y_train = []
x_test = []
y_test = []

for i in train_data:
    x_train.append(i[0])
    y_train.append(i[1])

for j in test_data:
    x_test.append(j[0])
    y_test.append(j[1])

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: overflow encountered in square
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: overflow encountered in multiply
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in multiply
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in subtract


[nan]


In [18]:
len(train_data)

5251

In [19]:
x=np.empty((5251,162))
y=np.empty((5251,1))

In [20]:
for i in range(5251):
    x[i,:]=train_data[i][0].reshape(1,-1)
    y[i,:]=train_data[i][1]

In [21]:
x_t=np.empty((500,162))
y_t=np.empty((500,1))

In [22]:
for i in range(500):
    x_t[i,:]=test_data[i][0].reshape(1,-1)
    y_t[i,:]=test_data[i][1]

In [23]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

LR = linear_model.LinearRegression()
LR.fit(x, y)
y_pred = LR.predict(x_t)
print(mean_squared_error(y_pred, y_t))

36.24358097096148
